In [3]:
import pandas as pd
import numpy as np

# Day 1

Fuel required to launch a given module is based on its mass. Specifically, to find the fuel required for a module, take its mass, divide by three, round down, and subtract 2.

In [25]:
df = pd.read_csv('day1.txt', sep='\n', header=None)

In [32]:
def fuel(mass):
    return int(mass / 3) - 2

np.sum(df.apply(lambda x: fuel(x[0]), axis=1))

3442987

In [40]:
def fuel_recursive(mass):
    
    total_fuel = fuel(mass)
    mass = total_fuel
    mass = fuel(mass)

    while (mass > 0):
        total_fuel += mass
        mass = fuel(mass)
    return total_fuel

In [43]:
np.sum(df.apply(lambda x: fuel_recursive(x[0]), axis=1))

5161601

# Day 2

In [76]:
def opcode(l, start_opcode, func):
    p1 = l[1 + start_opcode]
    p2 = l[2 + start_opcode]
    p3 = l[3 + start_opcode]
    l[p3] = func(l[p1], l[p2])

def apply_opcode(l, start_opcode):
    if l[start_opcode] == 1:
        opcode(l, start_opcode, lambda x, y: x + y)
    elif l[start_opcode] == 2:
        opcode(l, start_opcode, lambda x, y: x * y)
    

In [85]:
l = list(pd.read_csv('day2.txt', sep=',', header=None).values)[0]
l[1] = 12
l[2] = 2

start_opcode = 0
while (l[start_opcode] != 99):
    apply_opcode(l, start_opcode)
    start_opcode += 4
print(l[0])

7594646


In [88]:
answer = 19690720

def get_answer(noun, verb):
    l = list(pd.read_csv('day2.txt', sep=',', header=None).values)[0]
    l[1] = noun
    l[2] = verb

    start_opcode = 0
    while (l[start_opcode] != 99):
        apply_opcode(l, start_opcode)
        start_opcode += 4
    return l[0]

def find_noun_verb(answer):
    for noun in range(0, 99):
        for verb in range(0, 99):
            if get_answer(noun, verb) == answer:
                return (noun, verb)
            
noun, verb = find_noun_verb(answer)

print(noun, verb)
print(100 * noun + verb)

7594646

# Day 3

In [164]:
class Wire:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.coordinates = []
    
    def read_path(self, path):
        for c in path:
            self.read(c)
        return set(self.coordinates)
    
    def read(self, c):
        direction = c[0]
        n = int(c[1:])
        if direction == "R":
            self.move_horizontal(self.x + n, 1)
        elif direction == "L":
            self.move_horizontal(self.x - n, -1)
        elif direction == "U":
            self.move_vertical(self.y + n, 1)
        elif direction == "D":
            self.move_vertical(self.y - n, -1)
        
    def move_horizontal(self, end_x, direction):
        for i in range(self.x, end_x, direction):
            self.coordinates.append((i, self.y))
        self.x = end_x
    
    def move_vertical(self, end_y, direction):
        for i in range(self.y, end_y, direction):
            self.coordinates.append((self.x, i))
        self.y = end_y
    
    def steps_to_location(self, x, y):
        if (x, y) not in self.coordinates:
            print("Not found")
            return
        return self.coordinates.index((x, y))
    
    def __repr__(self):
        return f"{self.x}, {self.y}"

df = pd.read_csv('./day3.txt', header=None)
    
p1 = list(df.iloc[0, :])
p2 = list(df.iloc[1, :])    

w1 = Wire()
c1 = w1.read_path(p1)

w2 = Wire()
c2 = w2.read_path(p2)

intersections = c1.intersection(c2)
    

In [165]:
results = {}

for x, y in intersections:
    results[abs(x) + abs(y)] = (x, y)

sorted(results)[1]

1211

In [178]:
w1.coordinates.index((1, 0))
total_steps = {}

for x, y in intersections:
    total_steps[w1.steps_to_location(x, y) + w2.steps_to_location(x, y)] = (x, y)
    
sorted(total_steps)[1]

101386

# Day 4

* It is a six-digit number.
* The value is within the range given in your puzzle input.
* Two adjacent digits are the same (like 22 in 122345).
* Going from left to right, the digits never decrease; they only ever increase or stay the same (like 111123 or 135679).

Input: 264360-746325

In [327]:
puzzle_input = range(264360, 746325)

def six_digits(n):
    return n >= 10**5 and n < 10**6

def two_same_digits(n):
    return np.any([n[i] == n[i+1] for i in range(len(n) - 1)])

def increasing_digits(n):
    return np.all([n[i] <= n[i+1] for i in range(len(n) - 1)])

def part_a(i):
    if not six_digits(i):
        return False
    if not two_same_digits(str(i)):
        return False
    if not increasing_digits(str(i)):
        return False
    return True

result = [i for i in puzzle_input if part_a(i)]
        
len(result)

945

In [325]:
def only_two_same_digits(n):
    t = "x" + n + "x"
    return np.any([ (t[i] == t[i+1]) 
               and  (t[i+1] != t[i+2])
               and  (t[i-1] != t[i]) for i in range(1, len(n))])

def part_b(i):
    if not six_digits(i):
        return False
    if not only_two_same_digits(str(i)):
        return False
    if not increasing_digits(str(i)):
        return False
    return True

result = [i for i in puzzle_input if part_b(i)]

len(result)

617

# Day 5

* Opcode 3 takes a single integer as input and saves it to the position given by its only parameter. For example, the instruction 3,50 would take an input value and store it at address 50.
* Opcode 4 outputs the value of its only parameter. For example, the instruction 4,50 would output the value at address 50.

In [4]:
class Terminal:
    
    def __init__(self, filename: str):
        self.filename = filename
        self.l = []
        
        self.end_signal = 99
        self.pointer = 0
        
    def run(self, user_input):
        
        self.l = list(pd.read_csv(self.filename, sep=',', header=None).values)[0]
        
        while (self.l[self.pointer] != self.end_signal):  
            self.pointer += self.apply_opcode(user_input)
                
    def set_opcode(self, func, param_modes: list):
        param1 = self.get_param(self.l[1 + self.pointer], param_modes[0])
        param2 = self.get_param(self.l[2 + self.pointer], param_modes[1])
        param3 = self.l[3 + self.pointer]
        self.l[param3] = func(param1, param2)

    def get_param(self, p1, mode):    
        if mode == 0:
            return self.l[p1]
        return p1

    def opcode_input(self, user_input):
        p1 = self.l[self.pointer + 1]
        self.l[p1] = user_input

    def opcode_output(self, param_modes: list):
        p1 = self.l[self.pointer + 1]
        param1 = self.get_param(p1, param_modes[0])
        print(f"Test: {param1}")

    def parse_param_modes(self, param_str: str):

        param_str = list(reversed(param_str))
        opcode = int(param_str[0])
        param_modes = [0] * 3
        for i in range(0, len(param_str) - 2):
            param_modes[i] = int(param_str[i + 2])
        return opcode, param_modes

    def apply_opcode(self, user_input):
        opcode, param_modes = self.parse_param_modes(str(self.l[self.pointer]))
        if opcode == 1:
            self.set_opcode(lambda x, y: x + y, param_modes)
            return 4
        elif opcode == 2:
            self.set_opcode(lambda x, y: x * y, param_modes)
            return 4
        elif opcode == 3:
            self.opcode_input(user_input)
            return 2
        elif opcode == 4:
            self.opcode_output(param_modes)
            return 2
        else:
            print(f"Error: {self.l[self.pointer]}")
            return 0
        
test = Terminal('day5.txt')
test.run(user_input=1)

Test: 0
Test: 0
Test: 0
Test: 0
Test: 0
Test: 0
Test: 0
Test: 0
Test: 0
Test: 5346030


## Part b

* Opcode 5 is jump-if-true: if the first parameter is non-zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.
* Opcode 6 is jump-if-false: if the first parameter is zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.
* Opcode 7 is less than: if the first parameter is less than the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0.
* Opcode 8 is equals: if the first parameter is equal to the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0.

In [48]:
class Terminal:
    
    def __init__(self, filename: str):
        self.filename = filename
        self.l = []
        
        self.end_signal = 99
        self.pointer = 0
        
    def run(self, user_input):
        
        self.l = list(pd.read_csv(self.filename, sep=',', header=None).values)[0]
        
        while (self.l[self.pointer] != self.end_signal):  
            self.apply_opcode(user_input)
            
    def run_test(self, test_list, user_input):
        self.l = test_list
        
        while (self.l[self.pointer] != self.end_signal):  
            self.apply_opcode(user_input)
                
    def set_opcode(self, func, param_modes: list):
        param1 = self.get_param(1 + self.pointer, param_modes[0])
        param2 = self.get_param(2 + self.pointer, param_modes[1])
        param3 = self.l[3 + self.pointer]
        self.l[param3] = func(param1, param2)
        self.pointer += 4

    def get_param(self, location, mode):
        p1 = self.l[location]
        if mode == 0:
            return self.l[p1]
        return p1

    def opcode_input(self, user_input):
        p1 = self.l[self.pointer + 1]
        self.l[p1] = user_input
        self.pointer += 2

    def opcode_output(self, param_modes: list):
        param1 = self.get_param(self.pointer + 1, param_modes[0])
        print(f"Test: {param1}")
        self.pointer += 2
        
    def opcode_jump_if(self, param_modes: list, func):
        param1 = self.get_param(self.pointer + 1, param_modes[0])
        param2 = self.get_param(self.pointer + 2, param_modes[1])
        
        if func(param1):
            self.pointer = param2
        else:
            self.pointer += 3
            
    def opcode_compare(self, param_modes: list, func):
        param1 = self.get_param(self.pointer + 1, param_modes[0])
        param2 = self.get_param(self.pointer + 2, param_modes[1])
        param3 =  self.get_param(self.pointer + 3, param_modes[2])
        if func(param1, param2):
            self.l[param3] = 1
        else:
            self.l[param3] = 0
        self.pointer += 4

    def parse_param_modes(self, param_str: str):

        param_str = list(reversed(param_str))
        opcode = int(param_str[0])
        param_modes = [0] * 3
        for i in range(0, len(param_str) - 2):
            param_modes[i] = int(param_str[i + 2])
        return opcode, param_modes

    def apply_opcode(self, user_input):
        opcode, param_modes = self.parse_param_modes(str(self.l[self.pointer]))
        if opcode == 1:
            self.set_opcode(lambda x, y: x + y, param_modes)
        elif opcode == 2:
            self.set_opcode(lambda x, y: x * y, param_modes)
        elif opcode == 3:
            self.opcode_input(user_input)
        elif opcode == 4:
            self.opcode_output(param_modes)
        elif opcode == 5:
            self.opcode_jump_if(param_modes, lambda x: x != 0)
        elif opcode == 6:
            self.opcode_jump_if(param_modes, lambda x: x == 0)
        elif opcode == 7:
            self.opcode_compare(param_modes, lambda x, y: x < y)
        elif opcode == 8:
            self.opcode_compare(param_modes, lambda x, y: x == y)
        else:
            print(f"Error: {self.l[self.pointer]}")

terminal2 = Terminal('day5.txt')
l = [3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9]
terminal2.run(5)        

Test: 1998926
